In [4]:
from io_data import read_data, write_data,write_img
import cv2
import numpy as np
from scipy.stats import norm
from math import log, exp


In [5]:
data,img = read_data('../a1/4_noise.txt',True,save=True)
img.shape

(100, 100, 1)

In [45]:
class VI:
    def __init__(self,img):
        self.width = img.shape[0]
        self.height = img.shape[1]
        self.img = img

        self.mus = np.ones((img.shape[0],img.shape[1]))*0.5#bernouli
        self.J = 2 #coupling strength
        self.vari = 1
        
    def neighbours(self,x,y):
        n=[]
        if (not x==0):
            n.append((x-1,y))
        if (not x==self.width-1):
            n.append((x+1,y))
        if (not y==0):
            n.append((x,y-1))
        if (not y==self.height-1):
            n.append((x,y+1))
            
        return n

    def pairwisePotential(self,p1,p2):
        (x1,y1) = p1
        (x2,y2) = p2
        xs = self.img[x1][y1]
        xt = self.img[x2][y2]
        xs = 1*(xs>125)+(-1)*(xs<125)
        xt = 1*(xt>125)+(-1)*(xt<125)

        return (self.J*xs*xt,self.J*(-xs)*xt)
    
    def viprocess(self,name,n=1):

        elbolst = []
        counter = 0
        for i in range(n):
            elbo=0
            for x in range(self.img.shape[0]):
                for y in range(self.img.shape[1]):
                    if(counter%70000==0):
                        print(counter)
                    counter+=1
                    xs = self.img[x][y]
                    xs = 1*(xs>125)+(-1)*(xs<125)
                    
                    obv = self.img[x][y]
                    obv = 1*(obv>125)+(-1)*(obv<125)


#                     if(i==0):
#                         self.mus[x][y] = 0.9*(xs==1)+0.1*(xs==-1)
                    
                    pairwise = [self.pairwisePotential((x,y),p) for p in self.neighbours(x,y)]
                    firstPairwise = exp(sum([x[0] for x in pairwise]))
                    secondPairwise = exp(sum([x[1] for x in pairwise]))
                    pairwiseSum = firstPairwise+secondPairwise
                    firstPairwise/=pairwiseSum
                    secondPairwise/=pairwiseSum
                    #print('pair ',firstPairwise,secondPairwise)
                    
                    pxi = firstPairwise
                    pyilxi=exp(-log(norm(xs, self.vari).pdf(obv)))
                    pyixi=pxi*pyilxi
                    
                    pxineg = secondPairwise
                    pyilxineg=exp(-log(norm((-xs), self.vari).pdf(obv)))
                    pyixineg=pxineg*pyilxineg
                    
                    mu = self.mus[x][y] #mu == p
                   # l= mu*log(pyixi/mu) +(1-mu)*log(pyixineg/(1-mu))
                    
                    #l'=ln(p(yx=1)/mu)=ln(p(yx=-1)/(1-mu))=0 (1-mu)/mu == p(yx-1)/p(yx1)
                    up = pyixi
                    down = pyixineg
 
                    if(xs == 1):
                        up=pyixineg
                        down=pyixi
                    
                    ratio = up/down
                    newmu = 1/(ratio+1)
                    
                    self.mus[x][y] = newmu

                    
                  #  elbo += l
                    
            for x in range(self.img.shape[0]):
                for y in range(self.img.shape[1]):
                    if (np.random.random()>self.mus[x][y]):
                        self.img[x][y] = 0
                    else:
                        self.img[x][y] = 255 
                    
            elbolst.append(elbo)
            write_img(img,name+str(i))
        return elbolst
    
    

In [46]:
def VIDenoise(img,name,loops=100):
    vi = VI(img)
    return vi.viprocess(name,n=5),vi

In [47]:
data,img = read_data('../a1/1_noise.txt',True,save=True)
elbos,vi = VIDenoise(img, 'vi_noise1_',loops = img.shape[0]*img.shape[1]*5)
write_img(img,'1_result_vi')

data,img = read_data('../a1/2_noise.txt',True,save=True)
elbos,vi = VIDenoise(img, 'vi_noise2_',loops = img.shape[0]*img.shape[1]*5)
write_img(img,'2_result_vi')

data,img = read_data('../a1/3_noise.txt',True,save=True)
elbos,vi = VIDenoise(img, 'vi_noise3_',loops = img.shape[0]*img.shape[1]*5)
write_img(img,'3_result_vi')

data,img = read_data('../a1/4_noise.txt',True,save=True)
elbos,vi = VIDenoise(img, 'vi_noise4_',loops = img.shape[0]*img.shape[1]*5)
write_img(img,'4_result_vi')

0
70000
140000
210000
280000
350000
420000
490000
560000
630000
700000
770000
840000
910000
980000
0
70000
140000
210000
280000
350000
420000
490000
560000
630000
700000
770000
0
70000
140000
210000
280000
350000
420000
490000
560000
630000
700000
0


In [133]:
elbos

[-69848.82578015336]

In [15]:
log(0)

ValueError: math domain error